In [24]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [25]:
'''
import glob
import pickle
files = glob.glob("../../datasets/ant-*.csv", recursive = True)
globa = pd.concat(map(pd.read_csv, files))
pickle.dump( globa, open( "ant_master.p", "wb" ) )
'''

'\nimport glob\nimport pickle\nfiles = glob.glob("../../datasets/ant-*.csv", recursive = True)\ngloba = pd.concat(map(pd.read_csv, files))\npickle.dump( globa, open( "ant_master.p", "wb" ) )\n'

In [26]:
ant_master = pickle.load( open( "camel_master.p", "rb" ) )

In [27]:
ant_master

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,camel,1.0,org.apache.camel.component.quartz.QuartzExchange,4,2,0,6,8,6,2,...,0.0,0,0.896552,0.500000,0,0,5.500000,1,0.7500,0
1,camel,1.0,org.apache.camel.component.mina.MinaComponent,6,3,0,21,33,15,1,...,0.0,0,0.800000,0.500000,2,2,28.333333,1,0.6667,0
2,camel,1.0,org.apache.camel.processor.loadbalancer.Random...,2,3,0,3,7,1,0,...,0.0,0,0.833333,0.666667,1,1,11.000000,1,0.5000,0
3,camel,1.0,org.apache.camel.builder.ValueBuilder,26,1,1,10,47,0,5,...,1.0,1,0.000000,0.258242,0,0,8.038462,2,1.0000,0
4,camel,1.0,org.apache.camel.component.xmpp.XmppComponent,4,3,0,4,19,6,1,...,0.0,0,0.888889,0.375000,1,1,14.500000,1,0.5000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,camel,1.6,org.apache.camel.util.concurrent.CountingLatch,6,1,0,4,13,0,3,...,1.0,1,0.000000,0.444444,0,0,6.500000,1,0.8333,1
961,camel,1.6,org.apache.camel.component.mail.MailUtils,3,1,0,4,19,3,4,...,0.0,0,0.000000,0.333333,0,0,61.333333,9,5.6667,0
962,camel,1.6,org.apache.camel.component.uface.swing.SwingBr...,3,2,0,8,16,3,1,...,0.0,0,0.875000,0.750000,0,0,13.666667,1,0.3333,0
963,camel,1.6,org.apache.camel.component.jpa.QueryBuilder,12,1,3,8,25,46,6,...,0.0,1,0.000000,0.226190,0,0,7.416667,2,1.1667,0


In [28]:
test = ant_master[ant_master['version']==1.6]
test

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,camel,1.6,org.apache.camel.component.cxf.CxfComponent,5,3,0,7,10,0,1,...,1.0,1,0.921053,0.360000,1,2,7.400000,1,0.6000,0
1,camel,1.6,org.apache.camel.impl.DefaultExchangeFormatter,4,1,0,3,5,4,1,...,1.0,1,0.000000,0.500000,0,0,3.000000,1,0.5000,0
2,camel,1.6,org.apache.camel.component.jetty.JettyHttpComp...,20,4,0,26,95,144,2,...,1.0,0,0.727273,0.197368,4,5,20.300000,3,1.0000,0
3,camel,1.6,org.apache.camel.language.simple.SimpleLanguage,3,2,0,8,22,3,2,...,0.0,0,0.750000,0.666667,1,3,54.000000,15,5.3333,1
4,camel,1.6,org.apache.camel.util.ServiceHelper,8,1,0,25,20,22,22,...,1.0,0,0.000000,0.250000,0,0,20.875000,1,0.7500,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,camel,1.6,org.apache.camel.util.concurrent.CountingLatch,6,1,0,4,13,0,3,...,1.0,1,0.000000,0.444444,0,0,6.500000,1,0.8333,1
961,camel,1.6,org.apache.camel.component.mail.MailUtils,3,1,0,4,19,3,4,...,0.0,0,0.000000,0.333333,0,0,61.333333,9,5.6667,0
962,camel,1.6,org.apache.camel.component.uface.swing.SwingBr...,3,2,0,8,16,3,1,...,0.0,0,0.875000,0.750000,0,0,13.666667,1,0.3333,0
963,camel,1.6,org.apache.camel.component.jpa.QueryBuilder,12,1,3,8,25,46,6,...,0.0,1,0.000000,0.226190,0,0,7.416667,2,1.1667,0


In [29]:
train = ant_master[ant_master['version']!=1.6]
train

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,camel,1.0,org.apache.camel.component.quartz.QuartzExchange,4,2,0,6,8,6,2,...,0.000000,0,0.896552,0.500000,0,0,5.500000,1,0.7500,0
1,camel,1.0,org.apache.camel.component.mina.MinaComponent,6,3,0,21,33,15,1,...,0.000000,0,0.800000,0.500000,2,2,28.333333,1,0.6667,0
2,camel,1.0,org.apache.camel.processor.loadbalancer.Random...,2,3,0,3,7,1,0,...,0.000000,0,0.833333,0.666667,1,1,11.000000,1,0.5000,0
3,camel,1.0,org.apache.camel.builder.ValueBuilder,26,1,1,10,47,0,5,...,1.000000,1,0.000000,0.258242,0,0,8.038462,2,1.0000,0
4,camel,1.0,org.apache.camel.component.xmpp.XmppComponent,4,3,0,4,19,6,1,...,0.000000,0,0.888889,0.375000,1,1,14.500000,1,0.5000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867,camel,1.4,org.apache.camel.component.mail.MailConfiguration,47,1,0,7,81,829,5,...,0.857143,0,0.000000,0.183511,0,0,13.446809,9,1.4468,0
868,camel,1.4,org.apache.camel.util.ExpressionListComparator,3,1,0,4,9,1,1,...,1.000000,0,0.000000,0.500000,0,0,14.333333,2,1.0000,0
869,camel,1.4,org.apache.camel.model.language.PhpExpression,3,2,0,1,5,3,0,...,0.000000,0,0.956522,0.666667,1,2,3.000000,1,0.3333,0
870,camel,1.4,org.apache.camel.util.IntrospectionSupport,20,1,0,18,86,188,14,...,1.000000,0,0.000000,0.276316,0,0,35.900000,11,2.5000,1


In [30]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [31]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
0,4,2,0,6,8,6,2,5,4,2.000000,26,0.000000,0,0.896552,0.500000,0,0,5.500000,1,0.7500
1,6,3,0,21,33,15,1,21,2,2.000000,176,0.000000,0,0.800000,0.500000,2,2,28.333333,1,0.6667
2,2,3,0,3,7,1,0,3,1,2.000000,24,0.000000,0,0.833333,0.666667,1,1,11.000000,1,0.5000
3,26,1,1,10,47,0,5,5,24,0.080000,236,1.000000,1,0.000000,0.258242,0,0,8.038462,2,1.0000
4,4,3,0,4,19,6,1,4,3,2.000000,62,0.000000,0,0.888889,0.375000,1,1,14.500000,1,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867,47,1,0,7,81,829,5,3,45,0.939959,700,0.857143,0,0.000000,0.183511,0,0,13.446809,9,1.4468
868,3,1,0,4,9,1,1,3,3,0.000000,47,1.000000,0,0.000000,0.500000,0,0,14.333333,2,1.0000
869,3,2,0,1,5,3,0,1,3,2.000000,12,0.000000,0,0.956522,0.666667,1,2,3.000000,1,0.3333
870,20,1,0,18,86,188,14,4,12,0.894737,739,1.000000,0,0.000000,0.276316,0,0,35.900000,11,2.5000


In [32]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
0,5,3,0,7,10,0,1,7,4,0.250000,43,1.0,1,0.921053,0.360000,1,2,7.400000,1,0.6000
1,4,1,0,3,5,4,1,2,3,0.666667,17,1.0,1,0.000000,0.500000,0,0,3.000000,1,0.5000
2,20,4,0,26,95,144,2,26,13,0.842105,433,1.0,0,0.727273,0.197368,4,5,20.300000,3,1.0000
3,3,2,0,8,22,3,2,6,2,2.000000,165,0.0,0,0.750000,0.666667,1,3,54.000000,15,5.3333
4,8,1,0,25,20,22,22,3,6,0.571429,176,1.0,0,0.000000,0.250000,0,0,20.875000,1,0.7500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,6,1,0,4,13,0,3,2,6,0.000000,46,1.0,1,0.000000,0.444444,0,0,6.500000,1,0.8333
961,3,1,0,4,19,3,4,0,2,1.500000,201,0.0,0,0.000000,0.333333,0,0,61.333333,9,5.6667
962,3,2,0,8,16,3,1,7,2,2.000000,44,0.0,0,0.875000,0.750000,0,0,13.666667,1,0.3333
963,12,1,3,8,25,46,6,7,8,0.454545,102,0.0,1,0.000000,0.226190,0,0,7.416667,2,1.1667


In [33]:
y_train = train['bug']
y_test = test['bug']

In [34]:
y_train

0      0
1      0
2      0
3      0
4      0
      ..
867    0
868    0
869    0
870    1
871    0
Name: bug, Length: 1819, dtype: int64

In [35]:
y_test

0      0
1      0
2      0
3      1
4      1
      ..
960    1
961    0
962    0
963    0
964    0
Name: bug, Length: 965, dtype: int64

In [36]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [37]:
x_train_np

array([[ 4.        ,  2.        ,  0.        , ...,  5.5       ,
         1.        ,  0.75      ],
       [ 6.        ,  3.        ,  0.        , ..., 28.33333333,
         1.        ,  0.6667    ],
       [ 2.        ,  3.        ,  0.        , ..., 11.        ,
         1.        ,  0.5       ],
       ...,
       [ 3.        ,  2.        ,  0.        , ...,  3.        ,
         1.        ,  0.3333    ],
       [20.        ,  1.        ,  0.        , ..., 35.9       ,
        11.        ,  2.5       ],
       [ 2.        ,  5.        ,  0.        , ..., 11.        ,
         1.        ,  0.5       ]])

In [38]:
x_test_np

array([[ 5.        ,  3.        ,  0.        , ...,  7.4       ,
         1.        ,  0.6       ],
       [ 4.        ,  1.        ,  0.        , ...,  3.        ,
         1.        ,  0.5       ],
       [20.        ,  4.        ,  0.        , ..., 20.3       ,
         3.        ,  1.        ],
       ...,
       [ 3.        ,  2.        ,  0.        , ..., 13.66666667,
         1.        ,  0.3333    ],
       [12.        ,  1.        ,  3.        , ...,  7.41666667,
         2.        ,  1.1667    ],
       [ 4.        ,  1.        ,  0.        , ...,  5.25      ,
         1.        ,  0.75      ]])

In [39]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [40]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [41]:
train_x

array([[[ 4.        ,  2.        ,  0.        , ...,  5.5       ,
          1.        ,  0.75      ]],

       [[ 6.        ,  3.        ,  0.        , ..., 28.33333333,
          1.        ,  0.6667    ]],

       [[ 2.        ,  3.        ,  0.        , ..., 11.        ,
          1.        ,  0.5       ]],

       ...,

       [[ 3.        ,  2.        ,  0.        , ...,  3.        ,
          1.        ,  0.3333    ]],

       [[20.        ,  1.        ,  0.        , ..., 35.9       ,
         11.        ,  2.5       ]],

       [[ 2.        ,  5.        ,  0.        , ..., 11.        ,
          1.        ,  0.5       ]]])

In [42]:
test_y

array([ 0,  0,  0,  1,  1,  0,  0,  2,  0,  0,  3,  0,  0,  0,  1,  0,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  1,  1,  0,  2,  0,  0,  8,
        0,  0,  1,  0,  2,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  8,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,
        1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
        1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  0,  0,  0,  1,
        0,  0,  1,  1,  0,  0,  2,  0,  0,  0,  0,  0,  1,  0,  2,  0,  0,
        0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8,  0,
        1,  1,  0,  1,  1,  1,  0,  0,  4,  1,  2,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
        0,  8,  0, 18,  0,  0,  0,  5,  0,  0,  0,  0,  0,  5,  0,  0,  0,
        2, 11,  0,  3,  0

In [43]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0005) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [44]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])

Epoch 1/100
19/19 [==============================] - 5s 31ms/step - loss: 2.2455 - mse: 2.2455 - mae: 0.5035 - root_mean_squared_error: 1.4985 - mean_squared_logarithmic_error: 0.2628
Epoch 2/100
19/19 [==============================] - 0s 7ms/step - loss: 2.1045 - mse: 2.1045 - mae: 0.6253 - root_mean_squared_error: 1.4507 - mean_squared_logarithmic_error: 0.2338
Epoch 3/100
19/19 [==============================] - 0s 8ms/step - loss: 2.0339 - mse: 2.0339 - mae: 0.7685 - root_mean_squared_error: 1.4261 - mean_squared_logarithmic_error: 0.2656
Epoch 4/100
19/19 [==============================] - 0s 8ms/step - loss: 1.9922 - mse: 1.9922 - mae: 0.6942 - root_mean_squared_error: 1.4114 - mean_squared_logarithmic_error: 0.2328
Epoch 5/100
19/19 [==============================] - 0s 5ms/step - loss: 1.9446 - mse: 1.9446 - mae: 0.7168 - root_mean_squared_error: 1.3945 - mean_squared_logarithmic_error: 0.2444
Epoch 6/100
19/19 [==============================] - 0s 5ms/step - loss: 1.9047 - ms

19/19 [==============================] - 0s 6ms/step - loss: 0.9301 - mse: 0.9301 - mae: 0.4527 - root_mean_squared_error: 0.9644 - mean_squared_logarithmic_error: 0.1349
Epoch 89/100
19/19 [==============================] - 0s 5ms/step - loss: 0.9278 - mse: 0.9278 - mae: 0.4605 - root_mean_squared_error: 0.9632 - mean_squared_logarithmic_error: 0.1343
Epoch 90/100
19/19 [==============================] - 0s 5ms/step - loss: 0.9508 - mse: 0.9508 - mae: 0.4777 - root_mean_squared_error: 0.9751 - mean_squared_logarithmic_error: 0.1387
Epoch 91/100
19/19 [==============================] - 0s 5ms/step - loss: 0.9968 - mse: 0.9968 - mae: 0.4825 - root_mean_squared_error: 0.9984 - mean_squared_logarithmic_error: 0.1409
Epoch 92/100
19/19 [==============================] - 0s 6ms/step - loss: 0.9016 - mse: 0.9016 - mae: 0.4573 - root_mean_squared_error: 0.9495 - mean_squared_logarithmic_error: 0.1334
Epoch 93/100
19/19 [==============================] - 0s 6ms/step - loss: 1.0542 - mse: 1.054

In [45]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6007 (pid 27262), started 2:09:34 ago. (Use '!kill 27262' to kill it.)

In [46]:
score = model.evaluate(test_x, test_y)
score

31/31 [==============================] - 1s 2ms/step - loss: 3.9654 - mse: 3.9654 - mae: 0.7967 - root_mean_squared_error: 1.9913 - mean_squared_logarithmic_error: 0.3018


[3.9654312133789062,
 3.9654312133789062,
 0.7966642379760742,
 1.991339087486267,
 0.30181875824928284]